# 🎤 GPT-SoVITS + vTTS API 테스트 (Kaggle)

**목표**: GPT-SoVITS 음성 클로닝 TTS를 vTTS API로 테스트

---

## 📌 사전 요구사항
- Kaggle GPU 활성화 필수! (Settings → Accelerator → GPU T4 x2)
- 예상 시간: 설치 ~10-15분, 테스트 ~5분

## 1️⃣ 환경 설정 및 설치 (~10-15분)

In [ ]:
# GPT-SoVITS 설치 (시간이 오래 걸립니다 ~10-15분)
import os
import subprocess
import sys

print("🔧 GPT-SoVITS 환경 설정 중...")
print("   ⚠️ 이 과정은 10-15분 정도 소요됩니다.\n")

# 1. numpy + onnxruntime 완전 재설치 (순서 중요!)
print("   [1/7] numpy + ONNX Runtime 재설치 (호환성 보장)...")
!pip uninstall numpy onnxruntime onnxruntime-gpu -y -q
!pip install -q "numpy>=1.24.0,<2.0.0" onnxruntime-gpu

# 2. vTTS + Supertonic 설치 (GPU 자동 지원, numpy는 재설치 안함)
print("   [2/7] vTTS + Supertonic 설치 (GPU 자동 지원)...")
!pip install -q "vtts[supertonic] @ git+https://github.com/bellkjtt/vTTS.git" --no-deps
!pip install -q fastapi uvicorn httpx "pydantic>=2.0" pydantic-settings loguru soundfile librosa huggingface-hub

# 3. CUDA 지원 확인
print("   [3/7] CUDA 지원 확인...")
import onnxruntime as ort
providers = ort.get_available_providers()
print(f"      ONNX Providers: {providers}")
if "CUDAExecutionProvider" in providers:
    print("      🎉 CUDA 지원 활성화!")
else:
    print("      ⚠️ CUDA 미지원 (CPU 모드)")

# 4. GPT-SoVITS 저장소 클론
print("   [4/7] GPT-SoVITS 저장소 클론...")
GPT_SOVITS_PATH = os.path.expanduser("~/.vtts/GPT-SoVITS")
os.makedirs(os.path.dirname(GPT_SOVITS_PATH), exist_ok=True)

if not os.path.exists(GPT_SOVITS_PATH):
    !git clone --depth 1 https://github.com/RVC-Boss/GPT-SoVITS.git {GPT_SOVITS_PATH}
else:
    print(f"      GPT-SoVITS already exists: {GPT_SOVITS_PATH}")

# 5. GPT-SoVITS requirements 설치
print("   [5/7] GPT-SoVITS 의존성 설치 (오래 걸림)...")
!pip install -q -r {GPT_SOVITS_PATH}/requirements.txt

# 6. NLTK 데이터 다운로드 (GPT-SoVITS 영어 처리용)
print("   [6/7] NLTK 데이터 다운로드...")
import nltk
nltk.download('averaged_perceptron_tagger_eng', quiet=True)
nltk.download('cmudict', quiet=True)  # 발음 사전 (추가)
print("      NLTK 데이터 다운로드 완료!")

# 7. 환경변수 설정
print("   [7/7] 환경변수 설정...")
os.environ["GPT_SOVITS_PATH"] = GPT_SOVITS_PATH

print(f"\n✅ GPT-SoVITS 설치 완료!")
print(f"   경로: {GPT_SOVITS_PATH}")

## 2️⃣ 참조 오디오 생성 (Supertonic 사용)

In [ ]:
# 참조 오디오 생성 (GPT-SoVITS는 참조 오디오가 필수!)
from vtts.engines.supertonic import SupertonicEngine
from vtts.engines.base import TTSRequest
import IPython.display as ipd
from pathlib import Path
import soundfile as sf

print("🎙️ 참조 오디오 생성 중 (Supertonic 사용)...\n")

# Supertonic 엔진 로드
engine = SupertonicEngine(model_id="Supertone/supertonic-2", device="cuda")
engine.load_model()

# 참조 오디오 저장 디렉토리
ref_audio_dir = Path("./reference_audio")
ref_audio_dir.mkdir(exist_ok=True)

# 한국어 참조 오디오 생성
try:
    ko_request = TTSRequest(
        text="안녕하세요, 저는 음성 클로닝을 위한 참조 음성입니다.",
        language="ko",
        voice="F1",
        speed=1.0,
        extra_params={"total_steps": 10}
    )
    ko_output = engine.synthesize(ko_request)
    ko_path = ref_audio_dir / "reference_ko.wav"
    sf.write(ko_path, ko_output.audio, ko_output.sample_rate)
    print(f"✅ 한국어 참조 오디오: {ko_path}")
    print(f"   내용: {ko_request.text}")
except Exception as e:
    print(f"⚠️ 참조 오디오 생성 실패: {e}")
    print("   직접 오디오 파일을 업로드하세요!")
    ko_path = None

# 영어 참조 오디오 생성
try:
    en_request = TTSRequest(
        text="Hello, this is a reference audio for voice cloning.",
        language="en",
        voice="F1",
        speed=1.0,
        extra_params={"total_steps": 10}
    )
    en_output = engine.synthesize(en_request)
    en_path = ref_audio_dir / "reference_en.wav"
    sf.write(en_path, en_output.audio, en_output.sample_rate)
    
    # 엔진 언로드 (메모리 절약)
    engine.unload_model()
    
    print(f"\n✅ 영어 참조 오디오: {en_path}")
    print(f"   내용: {en_request.text}")
except Exception as e:
    print(f"⚠️ 영어 참조 오디오 생성 실패: {e}")
    en_path = None

print(f"\n📁 참조 오디오가 저장되었습니다: {ref_audio_dir.absolute()}")

## 3️⃣ vTTS 서버 시작 (GPT-SoVITS)

In [ ]:
# GPT-SoVITS 서버 시작
import subprocess
import time
import httpx

DEVICE = "cuda"  # 또는 "cpu"
PORT = 8000

print(f"🚀 vTTS 서버 시작 중... (Device: {DEVICE}, Port: {PORT})")
print("   이 과정은 1-2분 정도 소요됩니다 (모델 다운로드 포함)\n")

# 서버 시작 (백그라운드, 로그 파일로 출력)
log_file = open("gptsovits_server.log", "w")
server_process = subprocess.Popen(
    ["vtts", "serve", "kevinwang676/GPT-SoVITS-v3",
     "--host", "0.0.0.0",
     "--port", str(PORT),
     "--device", DEVICE],
    stdout=log_file,
    stderr=subprocess.STDOUT
)

# 서버 헬스체크
print("⏳ 서버 시작 대기 중...")
for i in range(120):  # 최대 2분 대기
    try:
        response = httpx.get(f"http://localhost:{PORT}/health", timeout=5.0)
        if response.status_code == 200:
            print(f"\n✅ 서버 시작 완료! (http://localhost:{PORT})")
            print(f"   서버 상태: {response.json()}")
            break
    except:
        pass
    time.sleep(1)
    if i % 10 == 0:
        print(f"   대기 중... {i}초")
else:
    print("\n❌ 서버 시작 실패!")
    print("\n📋 서버 로그 (마지막 100줄):")
    log_file.close()
    with open("gptsovits_server.log", "r") as f:
        lines = f.readlines()
        print("".join(lines[-100:]))
    raise RuntimeError("Server failed to start")

log_file.close()

## 🔍 디버깅: 변수 및 서버 로그 확인

In [ ]:
# 디버깅: 변수 확인
print(f"🔍 디버깅 정보:")
print(f"   ko_path 변수 존재: {'ko_path' in globals()}")
if 'ko_path' in globals():
    print(f"   ko_path 값: {ko_path}")
    print(f"   ko_path 존재: {ko_path.exists() if ko_path else 'N/A'}")
else:
    print("   ⚠️ ko_path 변수가 없습니다! Cell 4를 먼저 실행하세요.")

# 서버 로그 확인 (마지막 50줄)
print(f"\n📋 서버 로그 (마지막 50줄):")
try:
    with open("gptsovits_server.log", "r") as f:
        lines = f.readlines()
        print("".join(lines[-50:]))
except FileNotFoundError:
    print("   로그 파일이 없습니다.")

## 4️⃣ 음성 클로닝 테스트 (한국어)

In [ ]:
# GPT-SoVITS 음성 클로닝 테스트 (한국어)
from vtts import VTTSClient
import IPython.display as ipd

client = VTTSClient(base_url="http://localhost:8000")

print("🎙️ GPT-SoVITS 음성 클로닝 테스트 (한국어)\n" + "="*60)

if 'ko_path' in globals() and ko_path and ko_path.exists():
    # 한국어 음성 클로닝 (전체 파라미터 사용 예제)
    audio = client.tts(
        text="이것은 GPT-SoVITS 음성 클로닝 테스트입니다. 참조 음성을 기반으로 새로운 문장을 생성합니다.",
        model="kevinwang676/GPT-SoVITS-v3",
        language="ko",
        reference_audio=str(ko_path.absolute()),  # 절대 경로 사용
        reference_text="안녕하세요, 저는 음성 클로닝을 위한 참조 음성입니다.",
        # 🎛️ 품질 조절 파라미터 (선택)
        speed=1.0,             # 속도 (0.5-2.0, 기본: 1.0)
        top_k=15,              # Top-K 샘플링 (기본: 15)
        top_p=1.0,             # Top-P 샘플링 (기본: 1.0)
        temperature=1.0,       # 다양성 (0.1-2.0, 낮을수록 안정적, 기본: 1.0)
        sample_steps=32,       # 샘플링 스텝 (기본: 32, 높을수록 품질↑)
        seed=-1                # 시드 (-1: 랜덤, 고정값: 재현 가능)
    )
    
    print(f"✅ 음성 생성 완료!")
    print(f"   Sample rate: {audio.sample_rate}Hz")
    print(f"   Duration: {len(audio.audio)/audio.sample_rate:.2f}s")
    
    # 재생
    display(ipd.Audio(audio.audio, rate=audio.sample_rate))
    
    # 저장
    audio.save("gptsovits_cloning_ko.wav")
    print("   Saved: gptsovits_cloning_ko.wav")
else:
    print("❌ 참조 오디오가 없습니다. Cell 3를 먼저 실행하세요!")

## 🎛️ GPT-SoVITS 파라미터 가이드

**품질 조절 파라미터:**

| 파라미터 | 범위 | 기본값 | 설명 |
|---------|------|--------|------|
| `speed` | 0.5-2.0 | 1.0 | 음성 속도 (낮을수록 느림) |
| `top_k` | 1-100 | 15 | Top-K 샘플링 (높을수록 다양) |
| `top_p` | 0.0-1.0 | 1.0 | Top-P 샘플링 (낮을수록 안정) |
| `temperature` | 0.1-2.0 | 1.0 | 생성 다양성 (낮을수록 보수적) |
| `sample_steps` | 1-100 | 32 | 샘플링 스텝 (높을수록 품질↑ 속도↓) |
| `seed` | -1 or int | -1 | 랜덤 시드 (-1: 랜덤, 고정값: 재현) |

**추천 설정:**

- **고품질/안정적**: `temperature=0.7`, `top_p=0.9`, `sample_steps=40`
- **빠른 생성**: `sample_steps=16`, `top_k=10`
- **다양한 결과**: `temperature=1.2`, `top_k=30`

## 5️⃣ 음성 클로닝 테스트 (영어)

In [ ]:
# GPT-SoVITS 음성 클로닝 테스트 (영어)
print("🎙️ GPT-SoVITS 음성 클로닝 테스트 (영어)\n" + "="*60)

if 'en_path' in globals() and en_path and en_path.exists():
    # 영어 음성 클로닝 (절대 경로 사용)
    audio = client.tts(
        text="This is a voice cloning test using GPT-SoVITS. It generates new sentences based on the reference voice.",
        model="kevinwang676/GPT-SoVITS-v3",
        language="en",
        reference_audio=str(en_path.absolute()),  # 절대 경로 사용
        reference_text="Hello, this is a reference audio for voice cloning."
    )
    
    print(f"✅ 음성 생성 완료!")
    print(f"   Sample rate: {audio.sample_rate}Hz")
    print(f"   Duration: {len(audio.audio)/audio.sample_rate:.2f}s")
    
    # 재생
    display(ipd.Audio(audio.audio, rate=audio.sample_rate))
    
    # 저장
    audio.save("gptsovits_cloning_en.wav")
    print("   Saved: gptsovits_cloning_en.wav")
else:
    print("❌ 참조 오디오가 없습니다. Cell 3를 먼저 실행하세요!")

## 6️⃣ HTTP API 직접 호출 테스트

In [ ]:
# HTTP API 직접 호출
import httpx
import json
import IPython.display as ipd

print("🌐 HTTP API 직접 호출 테스트\n" + "="*60)

if 'ko_path' in globals() and ko_path and ko_path.exists():
    # TTS API 호출 (전체 파라미터 예제)
    payload = {
        "model": "kevinwang676/GPT-SoVITS-v3",
        "input": "HTTP API를 통한 음성 클로닝 테스트입니다. 다양한 파라미터로 품질을 조절할 수 있습니다.",
        "language": "ko",
        "reference_audio": str(ko_path.absolute()),  # 절대 경로 사용
        "reference_text": "안녕하세요, 저는 음성 클로닝을 위한 참조 음성입니다.",
        "response_format": "wav",
        # 🎛️ 품질 조절 파라미터
        "speed": 1.0,
        "top_k": 20,           # 더 넓은 샘플링 (더 다양한 결과)
        "top_p": 0.9,          # 약간 제한된 샘플링 (더 안정적)
        "temperature": 0.8,    # 낮은 temperature (더 보수적)
        "sample_steps": 40,    # 더 많은 스텝 (더 높은 품질)
        "seed": 42             # 고정 시드 (재현 가능)
    }
    
    response = httpx.post(
        "http://localhost:8000/v1/audio/speech",
        json=payload,
        timeout=60.0
    )
    
    if response.status_code == 200:
        print("✅ HTTP API 호출 성공!")
        print(f"   Status: {response.status_code}")
        print(f"   Content-Type: {response.headers.get('content-type')}")
        print(f"   Size: {len(response.content)} bytes")
        
        # 저장
        with open("http_api_gptsovits.wav", "wb") as f:
            f.write(response.content)
        print("   Saved: http_api_gptsovits.wav")
        
        # 재생
        display(ipd.Audio(response.content, rate=32000))
    else:
        print(f"❌ Error: {response.status_code}")
        print(response.text)
else:
    print("❌ 참조 오디오가 없습니다!")

## 7️⃣ Cleanup (서버 종료)

In [ ]:
# 서버 종료
if 'server_process' in globals():
    print("🛑 서버 종료 중...")
    server_process.terminate()
    server_process.wait()
    print("✅ 서버 종료 완료!")
else:
    print("⚠️ 실행 중인 서버가 없습니다.")